# 問題1：売上予測


## <font color='#3E1485'>**M5 Forecasting - Accuracy Competition**</font>


### <font color='#9271C3'>背景</font>

ウォルマートは米国最大級のスーパーマーケットである。同社が直面する主な課題の1つは、どの商品を、いつ、どのくらいの量仕入れるかを決定することである。間違った商品を仕入れると、潜在的な売上を逃し、在庫コストの増加につながる。

青果物のような特定の商品の需要は、収穫期やその商品の供給量と連動している場合がある。そのような商品の場合、供給が徐々に変化するにつれて、需要もスムーズに変化することが期待できる。しかし、他の商品では同じ傾向は見られず、休日や文化的行事によって需要が急激に変化することもある。商品需要の傾向を把握し予測することは、スーパーマーケットが適切な量の商品を仕入れ、損失を最小限に抑えながら利益を最大化するのに役立つ。

### <font color='#9271C3'>タスク</font>

ウォルマートのスーパーマーケットにおける様々な品目の過去の売上データが与えられたとき、個々の店舗における来月の売上を予測しなさい。

### <font color='#9271C3'>データ</font>

* `calendar.csv` - 商品が販売される日付に関する情報を含む。
* `sample_submission.csv` - 投稿用の正しいフォーマット。
* `sell_prices.csv` - 各店舗で販売された商品の価格と日付に関する情報が含まれている。
* `sales_train_evaluation.csv` - [d_1 - d_1941]` 日間における、各商品と各店舗の過去の日次販売個数データを格納。
* `sales_train_validation.csv` - [d_1 - d_1913]`日の各商品と店舗の過去の日次販売台数データを含む。**このファイルは使用しないでください。

\* 当初のコンペティションは2つのステージで構成されていた。第1ステージでは、1日目から1913日目までのデータが発表され、1914日目から1941日目までの予測に基づいて得点が決定された。第 2 段階では、1914 年～1941 年の日数のデータが発表され、1942 年～1969 年の日数の予測に基づ いて得点が決定された。ここでは簡単のため、`sales_train_evaluation.csv`のみを使用して、このコンペティションのセカンドステージをシミュレートする。

### <font color='#9271C3'>お役立ちリンク集</font>

[コンペHP](https://www.kaggle.com/c/m5-forecasting-accuracy)

[Kaggleコードリソース（日本語）](https://www.kaggle.com/competitions/m5-forecasting-accuracy/code?competitionId=18599&sortBy=voteCount&searchQuery=%E6%97%A5%E6%9C%AC%E8%AA%9E)

[主催者からの情報（英語）](https://mofc.unic.ac.cy/m5-competition/)

[1位解答](https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/163684)

[2位解答](https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/164599)

[3位解答](https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/164374)

## <font color='#3E1485'>**セットアップ**</font>

以下のセルは必要なデータをダウンロードし、ノートブックで使用する環境を設定します。

### <font color='#9271C3'>データセットをダウンロードする</font>

Kaggleはコンペティションと簡単にやり取りできるAPIを提供しています。私たちはこのAPIを使って自動的にデータをダウンロードし、予測をアップロードします。

このAPIを使用する最初のステップは、自分のユーザーとして認証することです。APIトークンはユーザー名とKaggleが生成したキーを含むファイルです。トークンはアカウントページからダウンロードすることができ、通常 `kaggle.json` と呼ばれます。APIトークンは、あなたのユーザーとしてAPIにアクセスするために必要なものなので、個人のGoogle Driveフォルダに安全に保管してください。

このノートブックはGoogle Driveフォルダ内の`kaggle.json`というKaggle APIトークンを検索します。トークンをGoogle Driveに置いたことを確認し、プロンプトが表示されたらこのノートブックがトークンにアクセスすることを許可してください。

認証後、参加したすべてのコンペティションにアクセスできます。データのダウンロードにエラーが発生した場合は、Kaggle API トークンが有効であること、コンペティションのルールに同意していることを確認してください。

In [ ]:
from google.colab import drive
import os
import json

drive.mount("/content/drive", force_remount=True)
fin = open("/content/drive/MyDrive/kaggle.json", "r")
json_data = json.load(fin)
fin.close()
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

In [ ]:
%%bash
kaggle competitions download -c m5-forecasting-accuracy --force
if [ $? -ne 0 ]; then
    echo "データのダウンロードに問題があったようです。"
    echo "競技規則に同意し、APIキーが有効であることを確認してください。"
else
    mkdir -p /content/kaggle
    unzip -o /content/m5-forecasting-accuracy.zip -d /content/kaggle
fi
wget -q -P /tmp https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip
unzip -u /tmp/NotoSansCJKjp-hinted.zip -d /usr/share/fonts/NotoSansCJKjp

### <font color='#9271C3'>計算環境</font>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast

font_path = '/usr/share/fonts/NotoSansCJKjp/NotoSansMonoCJKjp-Regular.otf'
matplotlib.font_manager.fontManager.addfont(font_path)
prop = matplotlib.font_manager.FontProperties(fname=font_path)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()
os.chdir('/content/kaggle')

## <font color='#3E1485'>**視覚化**</font>

いくつかの異なるファイルがあるので、視覚化は各ファイルにどのような種類のデータがあるのかを知るのに特に役立つでしょう。

### <font color='#9271C3'>sales_train_evaluation.csv</font>

In [ ]:
sales = pd.read_csv('sales_train_evaluation.csv')
sales.sample(10)

このファイルには `item_id` で指定されたアイテムの販売履歴が含まれる。アイテムには関連する部門とカテゴリが `dept_id` と `cat_id` に格納されている。さらに、場所に関する情報が `state_id` に、特定の店舗に関する情報が `store_id` に格納されている。残りの `d_x` というパターンにマッチする列は、`x` 日の売上数を示している。データは 2011-01-29 から始まるので、`d_1` には2011-01-29の売上が格納される。

トレーニングデータは3つのカテゴリーに大別される：
- 食べ物
- 趣味
- 家庭用品

以下のプロットは、ほとんどの売上が食品であることを示している。

In [ ]:
total_sales = sales.copy()
day_cols = [col for col in sales.columns if col.startswith('d_')]
total_sales['total'] = total_sales[day_cols].sum(axis=1)
total_sales = total_sales.groupby('cat_id', as_index=False).agg({'total': 'sum'})
ax = sns.barplot(total_sales, x='cat_id', y='total');

これは時系列データであり、我々の予測は予測であるため、データが時間とともにどのように変化するかを意識する必要がある。

In [ ]:
sales_long = sales.groupby('cat_id', as_index=False).agg({col: 'sum' for col in day_cols})
sales_long = sales_long.melt(id_vars=['cat_id'], var_name='day', value_name='sales')
sales_long['day'] = sales_long['day'].str.replace('d_', '').astype(int)
ax = sns.lineplot(sales_long, x='day', y='sales', hue='cat_id')
ax.grid()
ax.legend();

この図を見ると、すべての売上高はおおむね増加しているが、売上高が停滞、あるいは減少している時期もある。

時系列を分析するための一般的な手法のひとつに、トレンドと季節的要素でデータをモデル化する分解がある。このデータでは、トレンドは明らかに上向きだが、売上高の変動も見られる。これらの変動は、周期的、あるいは季節的な何らかの行動によるものかもしれない。

分解を行うには、`statsmodels` パッケージを使用します。

In [ ]:
sales_long = sales.groupby('cat_id', as_index=False).agg({col: 'sum' for col in day_cols})
sales_long = sales_long.melt(id_vars=['cat_id'], var_name='day', value_name='sales')
sales_long['day'] = sales_long['day'].str.replace('d_', '').astype(int)
sales_long = sales_long.pivot(index='day', columns='cat_id', values='sales')
sales_long = sales_long.sort_index()
sales_long = sales_long.set_index(pd.to_datetime(pd.read_csv('calendar.csv')['date'].values[:1941]))
sales_long = sales_long[sales_long.index.year==2012]
stl = STL(sales_long['FOODS'], period=30)
res = stl.fit()
ax = res.plot()
fig = ax.get_figure()
fig.set_size_inches(14, 7)
ax.tight_layout()

上の図は、季節期間を30日に設定した2012年の`FOODS`の分解である。トレンドと季節を分けることで、それぞれが元のデータにどのように寄与しているかを明確に見ることができる。1つの観察点は、シーズンの初め、つまりこの場合は月の初めに、売上が一般的に高くなることである。

### <font color='#9271C3'>calendar.csv</font>

In [ ]:
calendar = pd.read_csv('calendar.csv')
calendar

このファイルにはデータセットの各日に関する情報が含まれており、 `d_1` のような日を `2011-01-29` のような実際の日付に変換するのに使うことができる。データセットの開始日と終了日は1年単位で揃っていないことに注意しよう。これを下図に示す。

In [ ]:
sns.catplot(data=calendar, x='year', kind='count', color='tab:blue');

このファイルには、特別なイベントに関する情報も含まれている。特別なイベントは比較的頻繁に発生しないが、売上に影響を与える可能性がある。

In [ ]:
event_idx = calendar['event_name_1'].notna() | calendar['event_name_2'].notna()
n_events = event_idx.sum()
plt.bar(['イベントない日', 'イベントある日'], [calendar.shape[0] - n_events, n_events])

### <font color='#9271C3'>sell_prices.csv</font>

In [ ]:
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices.head()

このデータは、各店舗の様々な商品の価格を時系列で示したものである。価格は場所によって異なる場合があります。

In [ ]:
wm_yr_wk_to_date = calendar[['wm_yr_wk', 'date']].groupby('wm_yr_wk').min()
sell_prices_dt = sell_prices.groupby(['wm_yr_wk', 'item_id'], as_index=False)['sell_price'].agg('mean')
sell_prices_dt = sell_prices_dt.merge(wm_yr_wk_to_date, how='inner', on='wm_yr_wk')
sell_prices_dt['date'] = pd.to_datetime(sell_prices_dt['date'])

ax = sns.lineplot(sell_prices_dt[sell_prices_dt['item_id']=='FOODS_1_001'], x='date', y='sell_price', errorbar=None, label='FOODS_1_001')
ax = sns.lineplot(sell_prices_dt[sell_prices_dt['item_id']=='HOUSEHOLD_1_001'], x='date', y='sell_price', errorbar=None, label='HOUSEHOLD_1_001')
ax = sns.lineplot(sell_prices_dt[sell_prices_dt['item_id']=='HOBBIES_1_001'], x='date', y='sell_price', errorbar=None, label='HOBBIES_1_001')
ax.legend()
ax.figure.autofmt_xdate()

この図では3つの商品の価格を示している。一時的な売れ行きを示すと思われる、小規模で短期的な価格下落が見られる。また、長期的な価格の変化も見られるが、これは市場の需要や生産コストに対するメーカーの対応と考えられる。

## <font color='#3E1485'>**モデリング**</font>

### <font color='#9271C3'>分解と予測</font>

将来の売上を予測するために、上で紹介した時系列分解テクニックと、時系列予測によく使われるモデルであるARIMAモデルを組み合わせます。

単純なベースラインモデルとして、カテゴリーごとに全国的な売上高を予測する。このモデルは特定の商品の動向を無視し、立地も考慮しないため、あまり良いスコアは得られないかもしれません。

In [ ]:
sales_modeling = sales.groupby('cat_id', as_index=False).agg({col: 'sum' for col in sales.columns if col.startswith('d_')})
sales_modeling = sales_modeling.melt(id_vars=['cat_id'], var_name='day', value_name='sales')
sales_modeling['day'] = sales_modeling['day'].str.replace('d_', '').astype(int)
sales_modeling = sales_modeling.pivot(index='day', columns='cat_id', values='sales')
sales_modeling = sales_modeling.sort_index()
sales_modeling = sales_modeling.set_index(pd.to_datetime(pd.read_csv('calendar.csv')['date'].values[:1941]))
sales_modeling.index.freq = 'D'

forecast = {}
for cat_id in sales['cat_id'].unique():
    stlf = STLForecast(sales_modeling[cat_id], ARIMA, model_kwargs=dict(order=(1, 1, 0), trend="t"), period=30)
    stlf_res = stlf.fit()
    forecast[cat_id] = stlf_res.forecast(28)

In [ ]:
fig, ax = plt.subplots()
for i, cat_id in enumerate(sales_modeling.columns):
  cmap = matplotlib.color_sequences['tab10']
  ax.plot(sales_modeling[cat_id].iloc[-100:].index, sales_modeling[cat_id].iloc[-100:].values, color=cmap[i], label=cat_id)
  ax.plot(forecast[cat_id].iloc[-100:].index, forecast[cat_id].iloc[-100:].values, color=cmap[i], linestyle='dashed', label=f'{cat_id}予測')
  ax.figure.autofmt_xdate()
  ax.legend(ncols=3)

最後に、提出ファイルを作成し、そこに予測値を記入します。総売上高を予測するので、各店舗の平均売上高を得るために店舗数で割らなければなりません。

In [ ]:
def extract_cat_id(string):
    return string.split('_')[0]

def extract_split(string):
    return string.split('_')[-1]

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['cat_id'] = submission['id'].apply(extract_cat_id)
submission['split'] = submission['id'].apply(extract_split)

# fill validation with correct data
validation_idx = submission['id'].str.endswith('validation')
forecast_cols = [f'F{i}' for i in range(1, 29)]
d_1914_1941_cols = [f'd_{i}' for i in range(1914, 1942)]
submission.loc[validation_idx, forecast_cols] = sales.loc[submission.loc[validation_idx].index, d_1914_1941_cols].values

# fill evaluation with forecasted data
evaluation_idx = submission['id'].str.endswith('evaluation')
for cat_id, forecasted_sales in forecast.items():
    cat_idx = submission['cat_id'] == cat_id
    num_stores = sum(evaluation_idx & cat_idx)
    submission.loc[(evaluation_idx & cat_idx), forecast_cols] = (forecasted_sales / num_stores).values
submission.drop(columns=['cat_id', 'split']).to_csv('submission.csv', index=False)
submission

### <font color='#9271C3'>Kaggleへのアップロード</font>

予測はKaggleのAPIを通して直接提出されるため、ファイルを手動でダウンロードし、Kaggleのウェブサイトに再アップロードする必要はありません。

In [ ]:
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m "これはアップロードに添付されたメッセージである。"

[提出ページ](https://www.kaggle.com/competitions/m5-forecasting-accuracy/submissions)でスコアを確認する。

## <font color='#3E1485'>提案</font>

**異なる季節性期間**：時系列分解の実験では、一般的に月の初めに売上が高くなることが示されました。時系列分解に週単位を使用することで、他の傾向を示す可能性がある。

**店舗を個別にモデル化する**：このノートブックでは現在、各店舗の売上が同じであると仮定しています。ある店舗は他の店舗よりも一般的に売上が高い可能性が高い。

**より複雑なモデル**：ここで使われているモデルは比較的単純です。LSTMやコンペティションの勝者が使ったモデルなど、他のタイプのモデルを使ってみてください。

**追加データ**：現在のモデルは販売データしか使っていません。しかし、価格や休日などのデータもあり、予測を改善できるかもしれません。